### 고객 구매상품 관련 분석
    
    최초 작성일 : 20/03/02
    작성자 : 김은주
    
    작성내용 : 고객 구매상품 분석
    
               DB 內 PRDT_REVIEW 테이블 활용
               - 상품코드 기준 상품 대표 이미지(ex. 썸네일 이미지) 크롤링
               -> 이미지에서 대표 색상 추출하여 DB 추가
               
               유저 닉네임 별 주요 구매 색상 추출
               -> 

    수정내용 
        20/03/02

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='C:\Windows/Fonts/malgun.ttf').get_name()
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"] = False

import pymysql, math, time
from sklearn import preprocessing
import urllib

In [ ]:
#DB 연결 데이터 
config = {"host":"192.168.0.44", "user":"musinsa", "passwd":"1111", "db":"PR_TEST"}
conn = pymysql.connect(**config)
cursor = conn.cursor()

In [ ]:
sql = "SELECT *,round((A.L+A.A+A.S+A.C)/4, 5) as 'con_avg' FROM (SELECT b.BRD_NAME_ENG, PRODUCT, REVIEW, LIKES, LOOKBOOK/(SELECT sum(LOOKBOOK) FROM BRAND_INFO) as 'L', ARTICLE/(SELECT sum(ARTICLE) FROM BRAND_INFO) as 'A', STYLE/(SELECT sum(STYLE) FROM BRAND_INFO) as 'S', CONTENTS/(SELECT sum(CONTENTS) FROM BRAND_INFO) as 'C' FROM BRAND_INFO a, BRAND_LIST b WHERE a.BRD_NAME_ENG = b.BRD_NAME_ENG AND a.BRD_NAME_ENG != 'Disney' AND a.BRD_NAME_ENG != 'ABERCROMBIE & FITCH') A WHERE (A.L+A.A+A.S+A.C) != 0;"
cursor.execute(sql)
brand_info = pd.DataFrame(cursor, columns=["brand", "prdt", "review", "likes", "lookbook", "article", "style", "contents", "score"])

columns = ["prdt", "review", "likes", "lookbook", "article", "style", "contents", "score"]
for col in columns:
    brand_info[col] = brand_info[col].astype(float)

brand_info.info()